In [ ]:
from pykinect import nui
import numpy
import cv2
import math
from collections import deque
import time

def video_handler_function(frame):
    video = numpy.empty((480,640,4),numpy.uint8)
    frame.image.copy_bits(video.ctypes.data)
    cv2.imshow('KINECT Video Stream', video)
    
    
def detect_left_hand_up(skelPos):
#     print('----')
    if(skelPos[JointId.HandLeft].y > skelPos[JointId.Head].y):
        avg_arm_length= 0.43 
        scaling_factor= 255/avg_arm_length
        relative_distance_of_left_arm = skelPos[JointId.HandLeft].y - skelPos[JointId.Head].y
        scaled_reldist_Larm= math.floor(relative_distance_of_left_arm * scaling_factor)
        if(scaled_reldist_Larm > 255):
            string_reldist_Larm= 'B' + str(255)
        else:
            string_reldist_Larm= 'B' + str(int(scaled_reldist_Larm))
        send_to_com_port(string_reldist_Larm)
        print(string_reldist_Larm)
#         return(string_reldist_Larm)

        
def detect_right_hand_up(skelPos):
    if(skelPos[JointId.HandRight].y > skelPos[JointId.Head].y):
        avg_arm_length= 0.43 
        scaling_factor= 255/avg_arm_length
        relative_distance_of_right_arm = skelPos[JointId.HandRight].y - skelPos[JointId.Head].y
        scaled_reldist_Rarm= math.floor(relative_distance_of_right_arm * scaling_factor)
        if(scaled_reldist_Rarm > 255):
            string_reldist_Rarm= 'A' + str(255)
        else:
            string_reldist_Rarm= 'A' + str(int(scaled_reldist_Rarm))
        print(scaled_reldist_Rarm)
#         return(string_reldist_Rarm)
        send_to_com_port(string_reldist_Rarm)

def detect_arm_stretch(skelPos):
    if(skelPos[JointId.HandRight].x != 0 and skelPos[JointId.HandLeft].x!=0):
        if(skelPos[JointId.HandLeft].y > skelPos[JointId.HipCenter].y and skelPos[JointId.HandRight].y > skelPos[JointId.HipCenter].y):
#         print(skelPos[JointId.HandRight].x - skelPos[JointId.HandLeft].x)
            avg_dist_bw_arms= 1.47
            scaling_factor= 255/avg_dist_bw_arms
            measured_dist_bw_arms= abs(skelPos[JointId.HandRight].x - skelPos[JointId.HandLeft].x)
            scaled_dist_bw_arms= math.floor(measured_dist_bw_arms*scaling_factor)
            if(scaled_dist_bw_arms>255):
    #             return('d'+ str(255))
    #             print('d'+ str(255))
                send_to_com_port('d'+ str(255))
            elif(measured_dist_bw_arms>0.5):
        #         return('d'+str(scaled_dist_bw_arms))
                print('d'+str(int(scaled_dist_bw_arms)))
                send_to_com_port('d'+str(int(scaled_dist_bw_arms)))
# #         discrete-valued function:
# #         if(scaled_dist_bw_arms>245):
# #             return('d'+str(255))

def detect_left_foot_ahead(skelPos):
    if(skelPos[JointId.FootLeft].z!=0 and skelPos[JointId.Spine].z!=0):
#         print("position of foot: ", skelPos[JointId.FootLeft].z)
#         print("position of spine: ", skelPos[JointId.Spine].z)
        fraction_left_leg_movement= (skelPos[JointId.FootLeft].z - skelPos[JointId.Spine].z)/skelPos[JointId.Spine].z
#         print(fraction_left_leg_movement)
        if(abs(fraction_left_leg_movement) >= 0.16):
            print('D'+ str(255))
            send_to_com_port('D'+ str(255))
#             return('c'+ str(255))

def detect_right_foot_ahead(skelPos):
    if(skelPos[JointId.FootRight].z!=0 and skelPos[JointId.Spine].z!=0):
#         print("position of foot: ", skelPos[JointId.FootLeft].z)
#         print("position of spine: ", skelPos[JointId.Spine].z)
        fraction_right_leg_movement= (skelPos[JointId.FootRight].z - skelPos[JointId.Spine].z)/skelPos[JointId.Spine].z
#         print(fraction_left_leg_movement)
        if(abs(fraction_right_leg_movement) >= 0.16):
            print('C'+ str(255))
#             return('C'+ str(255))
            send_to_com_port('C'+ str(255))

                 

# def detect_clap(skelPos):
#     left_hand_rel_posX=  
#     if(skelPos[JointId.HandLeft].x == skelPos[JointId.HandRight].x and skelPos[JointId.HandLeft].y == skelPos[JointId.HandRight].y):
#         print("clap clap!")


    
# Reshape a numpy array 'a' of shape (n, x) to form shape((n - window_size), window_size, x))
# def rolling_window(a, window, step_size):
#     shape = a.shape[:-1] + (a.shape[-1] - window + 1 - step_size, window)
#     strides = a.strides + (a.strides[-1] * step_size,)
#     return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

# max_length_of_position_window = 60
# sliding_position_window = deque()
# max_length_of_velocity_window = max_length_of_position_window - 1
# max_length_of_acceleration_window = max_length_of_velocity_window - 1 
# sliding_velocity_window = []
# sliding_acceleration_window = []
# # 0 holds the -2 value
# # 1 holds the -1 value
# def detect_drum_beat(skelPos):
#     global sliding_position_window
#     global max_length_of_position_window
#     global sliding_velocity_window
#     global sliding_acceleration_window
#     if len(list(sliding_position_window)) < max_length_of_position_window:
#         print("Appending to list till it fills first")
#         sliding_position_window.append(skelPos[JointId.HandLeft].y)
#         print(sliding_position_window)
#     else:
#         sliding_position_window.append(skelPos[JointId.HandLeft].y)
#         temp = list(sliding_position_window)
#         sliding_velocity_window = [x - temp[i - 1] for i, x in enumerate(temp) if i > 0]
#         sliding_acceleration_window = [x - sliding_velocity_window[i - 1] for i, x in enumerate(sliding_velocity_window) if i > 0]
#         sliding_position_window.popleft()
# #         print("Position: ", len(sliding_position_window))
#         print("Velocity: ", sliding_velocity_window)
#         print("Accel: ", sliding_acceleration_window)
#     #     
k = 5
def skeleton_handler_function(frame):
    global k
    k -= 1
    if k == 0:
        k = 5
#         print('Resetting count.') 
        for index,data in enumerate(frame.SkeletonData):
    #         detect_drum_beat(data.SkeletonPositions)
    #         LEFT_ARM = (JointId.ShoulderCenter, JointId.ShoulderLeft, JointId.ElbowLeft, JointId.countWristLeft, JointId.HandLeft)
    #         left_arm_positions = [data.SkeletonPositions[x] for x in LEFT_ARM]

            #uncomment the following function calls to test the motion detections:

            detect_left_hand_up(data.SkeletonPositions)
            detect_right_hand_up(data.SkeletonPositions)
#             detect_clap(data.SkeletonPositions)
            detect_arm_stretch(data.SkeletonPositions)
            detect_left_foot_ahead(data.SkeletonPositions)
            detect_right_foot_ahead(data.SkeletonPositions)
           
        
        



    
kinect = nui.Runtime()
kinect.video_frame_ready += video_handler_function

kinect.skeleton_frame_ready += skeleton_handler_function
kinect.skeleton_engine.enabled=True


# kinect.video_stream.open(nui.ImageStreamType.Video,2,nui.ImageResolution.Resolution640x480,nui.ImageType.Color)
kinect.depth_stream.open(nui.ImageStreamType.Depth, 2, nui.ImageResolution.Resolution640x480, nui.ImageType.Depth)
kinect.video_stream.open(nui.ImageStreamType.Video, 2,nui.ImageResolution.Resolution640x480,nui.ImageType.Color)

cv2.namedWindow('KINECT Video Stream', cv2.WINDOW_AUTOSIZE)

while True:

    key = cv2.waitKey(1)
    if key == 27: break

kinect.close()
cv2.destroyAllWindows()


0.0


Unhandled exception in thread started by <bound method Runtime._event_thread of <pykinect.nui.Runtime object at 0x0000000015EDFD68>>


SerialException: WriteFile failed (WindowsError(5, 'Access is denied.'))

In [18]:
d = 51.0
print(int(d))

51


In [ ]:

def sliding_window(linearPositionsArr, windowSize):
    shape= 10
    strides= 5
    print(np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides))
    


In [6]:
!pip install numpy --upgrade

  Found existing installation: numpy 1.16.2
    Uninstalling numpy-1.16.2:
      Successfully uninstalled numpy-1.16.2


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
ERROR: Could not install packages due to an EnvironmentError: [Error 5] Access is denied: 'c:\\users\\himanshi\\miniconda3\\envs\\py2\\lib\\site-packages\\~umpy\\core\\_multiarray_tests.pyd'
Consider using the `--user` option or check the permissions.



In [12]:
!pip install queue

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
  ERROR: Could not find a version that satisfies the requirement queue (from versions: none)
ERROR: No matching distribution found for queue


In [1]:
from vpython import *

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# from vpython import *
import pykinect
from pykinect import nui
from pykinect.nui import JointId

class Skeleton:
    """Kinect skeleton represented as a VPython frame.
    """

    def __init__(self, f):
        """Create a skeleton in the given VPython frame f.
        """
        self.frame = f
        self.joints = [sphere(frame=f, radius=0.08, color=color.yellow)
                       for i in range(20)]
        self.joints[3].radius = 0.125
        self.bones = [cylinder(frame=f, radius=0.05, color=color.yellow)
                      for bone in _bone_ids]

    def update(self):
        """Update the skeleton joint positions in the depth sensor frame.
        Return true iff the most recent sensor frame contained a tracked
        skeleton.
        """
        updated = False
        for skeleton in _kinect.skeleton_engine.get_next_frame().SkeletonData:
            if skeleton.eTrackingState == nui.SkeletonTrackingState.TRACKED:

                # Move the joints.
                for joint, p in zip(self.joints, skeleton.SkeletonPositions):
                    joint.pos = (p.x, p.y, p.z)

                # Move the bones.
                for bone, bone_id in zip(self.bones, _bone_ids):
                    p1, p2 = [self.joints[id].pos for id in bone_id]
                    bone.pos = p1
                    bone.axis = p2 - p1
                updated = True
        return updated

def draw_sensor(f):
    """Draw 3D model of the Kinect sensor.
    Draw the sensor in the given (and returned) VPython frame f, with
    the depth sensor frame aligned with f.
    """
    box(frame=f, pos=(0, 0, 0), length=0.2794, height=0.0381, width=0.0635,
        color=color.blue)
    cylinder(frame=f, pos=(0, -0.05715, 0), axis=(0, 0.0127, 0), radius=0.0381,
             color=color.blue)
    cone(frame=f, pos=(0, -0.04445, 0), axis=(0, 0.01905, 0), radius=0.0381,
         color=color.blue)
    cylinder(frame=f, pos=(0, -0.05715, 0), axis=(0, 0.0381, 0), radius=0.0127,
             color=color.blue)
    cylinder(frame=f, pos=(-0.0635, 0, 0.03175), axis=(0, 0, 0.003),
             radius=0.00635, color=color.red)
    cylinder(frame=f, pos=(-0.0127, 0, 0.03175), axis=(0, 0, 0.003),
             radius=0.00635, color=color.red)
    cylinder(frame=f, pos=(0.0127, 0, 0.03175), axis=(0, 0, 0.003),
             radius=0.00635, color=color.red)
    text(frame=f, text='KINECT', pos=(0.06985, -0.00635, 0.03175),
         align='center', height=0.0127, depth=0.003)
    return f

# A bone is a cylinder connecting two joints, each specified by an id.
_bone_ids = [[0, 1], [1, 2], [2, 3], [7, 6], [6, 5], [5, 4], [4, 2],
             [2, 8], [8, 9], [9, 10], [10, 11], [15, 14], [14, 13], [13, 12],
             [12, 0], [0, 16], [16, 17], [17, 18], [18, 19]]

# Initialize and level the Kinect sensor.
_kinect = nui.Runtime()
_kinect.skeleton_engine.enabled = True
_kinect.camera.elevation_angle = 0

# if __name__ == '__main__':
#     draw_sensor(frame())
#     skeleton = Skeleton(frame(visible=False))
#     while True:
#         rate(30)
#         skeleton.frame.visible = skeleton.update()

In [7]:
import serial

In [8]:
ser = serial.Serial(port='COM5', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=0, rtscts=0)

In [9]:
ser.open()

SerialException: Port is already open.

In [10]:
def send_to_com_port(input):
    character = input[0]
    number = input[1:]
    b = bytes(bytearray([ord(character)])) + bytes(bytearray([int(number)]))
    ser.write(b)
    
# send_to_com_port('a91')